In [1]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt

from datetime import timedelta
from time import time

import pickle
import os


import edslab_prophet_dask as edd
import edslab_prophet as ed

pd.options.display.float_format = '{:.2f}'.format

In [2]:
# %load D:/BUDS_MID/__initdask__.py
"""
Created on Wed Feb 19 16:24:03 2020

@author: cmchico
"""
import dask.bag as db
from dask.distributed import Client, progress

client = Client(processes = True,n_workers=12)
client

Client Scheduler: tcp://127.0.0.1:53242 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 12 Cores: 12 Memory: 34.14 GB


In [3]:
path = 'D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/Actual/'
out = 'D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/Prophet Train/'

HP = pd.read_csv(path + 'Holidays_ALL.csv')
data = pd.read_pickle(path+'82 Aggregated TIDBR WDL.pkl')

data.info()

train_dates = pd.date_range(start='2017-01-01',end='2019-03-31')
val_dates = pd.DataFrame(pd.date_range(start='2019-01-01',end='2019-03-31').date,columns=['ds'])
dev_dates = pd.DataFrame(pd.date_range(start='2017-01-01',end='2018-12-31').date,columns=['ds'])

dev_length = sum(train_dates < pd.to_datetime('20181231'))
period = '1 days'
horizon = '14 days'

week = [0,2,3]
mon = [0,2,3,4,5,6]
hol = ['HP']
holidays=HP


data_train = data.loc[data.DATE.isin(train_dates)][['DATE','TID_BR','WDL_AMT_FIN']]
data_train.columns = ['ds','tid','y']

scenario =  pd.DataFrame(list(itertools.product(hol,week,mon)),columns=['hol','week','mon'])
scenario['model'] = scenario.hol + scenario.week.astype('str') +scenario.mon.astype(str)
scenario['hol_num'] = scenario['hol'].map({'HP':0,'OP':1})

print('\ntotal scenario: {}'.format(scenario.shape[0]))

# ticket = pd.read_csv(path+'Ticket Count 2017 2018.csv')
atms = data_train.tid.unique()
# atm2 = list(set(data_train.tid.unique())-set(atms))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 751436 entries, 0 to 751435
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATE         751436 non-null  datetime64[ns]
 1   TID_BR       751436 non-null  object        
 2   WDL_AMT      751436 non-null  float64       
 3   WDL_AMT_FIN  751436 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 22.9+ MB

total scenario: 18


In [4]:
from fbprophet import Prophet

In [5]:
dbweek = db.from_sequence([scenario.week[i] for i in range(scenario.shape[0])])
dbmon = db.from_sequence([scenario.mon[i] for i in range(scenario.shape[0])])

In [6]:
def train(series,holidays, dev_dates,val_dates,weekly=0,monthly=0):
    series = series.copy()
    #define model
    modelf = Prophet(interval_width = 0.95,
                    daily_seasonality = False,
                    weekly_seasonality = False,
                    yearly_seasonality = False,
                    holidays = holidays)
    if weekly != 0: modelf.add_seasonality(name='weeklyx', period = 7, fourier_order = weekly)
    if monthly != 0: modelf.add_seasonality(name='monthlyx', period = 30.5, fourier_order = monthly)
    
    modelf.fit(series[series.ds.isin(dev_dates.ds)])
    
    series['yhat']  = modelf.predict(series[['ds']])['yhat'].values.clip(min=0)
    series['se'] =(series.y - series.yhat)**2
    
    
    dev_rmse = np.sqrt(series[series.ds.isin(dev_dates.ds)].se.mean())
    val_rmse = np.sqrt(series[series.ds.isin(val_dates.ds)].se.mean())
    return [dev_rmse, val_rmse]

In [7]:
def par_scen(model,tid):
    df = pd.DataFrame([[tid,scenario.model[i]] + 
                    model[i] for i in range(scenario.shape[0])],
                      columns=['tid','model','dev_rmse','val_rmse'])
    df.to_csv(out+tid+'_train.csv')
    return df

In [8]:
start = time()
for atm in atms:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    dbmaster = db.map(train,series,holidays,dev_dates, val_dates,weekly=dbweek,monthly=dbmon)
    model = par_scen(dbmaster.compute(),atm)
    print(model)
    print(timedelta(seconds=time()-start))

         tid model  dev_rmse  val_rmse
0   91010033  HP00 298556.65 236843.07
1   91010033  HP02 286657.34 223539.27
2   91010033  HP03 285876.87 227278.71
3   91010033  HP04 282278.96 224213.21
4   91010033  HP05 281348.43 224981.58
5   91010033  HP06 280234.32 224369.64
6   91010033  HP20 284526.85 220275.22
7   91010033  HP22 271140.54 204802.42
8   91010033  HP23 270178.62 206300.36
9   91010033  HP24 265672.07 201459.97
10  91010033  HP25 264281.71 205522.12
11  91010033  HP26 262987.26 201923.17
12  91010033  HP30 283817.05 215938.36
13  91010033  HP32 270504.62 201542.42
14  91010033  HP33 269601.60 203761.57
15  91010033  HP34 265168.47 200221.14
16  91010033  HP35 263799.61 202260.66
17  91010033  HP36 262592.14 199621.23
0:00:55.678110
         tid model  dev_rmse  val_rmse
0   91010053  HP00 314248.61 359472.49
1   91010053  HP02 310255.48 349756.95
2   91010053  HP03 309665.42 350046.20
3   91010053  HP04 309434.58 349667.42
4   91010053  HP05 308601.79 350304.69
5   910100

         tid model  dev_rmse  val_rmse
0   91010145  HP00 131919.05 118055.94
1   91010145  HP02 127847.54 117586.00
2   91010145  HP03 127818.08 117658.34
3   91010145  HP04 127535.23 117255.42
4   91010145  HP05 127541.40 117410.91
5   91010145  HP06 126897.11 117190.56
6   91010145  HP20 117990.54 103906.44
7   91010145  HP22 113296.13 103500.23
8   91010145  HP23 113357.96 103529.32
9   91010145  HP24 113051.54 102872.67
10  91010145  HP25 113036.26 103337.25
11  91010145  HP26 112724.17 102933.42
12  91010145  HP30 116334.89 101730.50
13  91010145  HP32 111672.76 101210.83
14  91010145  HP33 111480.05 101353.27
15  91010145  HP34 111388.26 100659.67
16  91010145  HP35 111293.94 101088.29
17  91010145  HP36 111138.56 100711.83
0:10:37.054544
         tid model  dev_rmse  val_rmse
0   91010151  HP00 382068.88 448735.43
1   91010151  HP02 378673.03 450225.16
2   91010151  HP03 378300.35 450844.76
3   91010151  HP04 378142.93 450228.04
4   91010151  HP05 378437.11 448956.83
5   910101

         tid model  dev_rmse  val_rmse
0   91010301  HP00 644881.04 717609.92
1   91010301  HP02 634818.79 702487.14
2   91010301  HP03 634962.86 704610.60
3   91010301  HP04 634523.63 703597.03
4   91010301  HP05 633806.18 701824.37
5   91010301  HP06 632942.23 703683.85
6   91010301  HP20 500028.80 547546.69
7   91010301  HP22 485672.66 528748.97
8   91010301  HP23 484433.29 531629.86
9   91010301  HP24 483865.55 528311.62
10  91010301  HP25 483212.63 532431.90
11  91010301  HP26 483420.09 531247.52
12  91010301  HP30 489210.93 539694.24
13  91010301  HP32 475000.95 522591.99
14  91010301  HP33 474009.36 524363.59
15  91010301  HP34 473022.52 522286.63
16  91010301  HP35 473644.46 522948.63
17  91010301  HP36 472796.11 523557.91
0:20:14.770033
         tid model  dev_rmse  val_rmse
0   91010320  HP00 761314.18 743898.65
1   91010320  HP02 729287.30 698362.29
2   91010320  HP03 729762.39 694349.46
3   91010320  HP04 727264.80 684751.57
4   91010320  HP05 726976.57 687137.26
5   910103

         tid model  dev_rmse  val_rmse
0   91010443  HP00 276791.28 329222.21
1   91010443  HP02 274463.68 327660.39
2   91010443  HP03 274289.26 326535.68
3   91010443  HP04 273931.29 327256.06
4   91010443  HP05 274038.14 327110.87
5   91010443  HP06 273411.01 323468.62
6   91010443  HP20 248721.39 314346.21
7   91010443  HP22 245986.41 312469.40
8   91010443  HP23 245826.60 311374.39
9   91010443  HP24 245461.22 311820.59
10  91010443  HP25 245474.21 312056.02
11  91010443  HP26 245437.63 311011.89
12  91010443  HP30 241451.10 307606.99
13  91010443  HP32 238742.97 305923.28
14  91010443  HP33 238343.61 304872.26
15  91010443  HP34 238065.41 304986.48
16  91010443  HP35 238008.58 305260.80
17  91010443  HP36 237959.57 304153.70
0:29:50.247110
         tid model  dev_rmse  val_rmse
0   91010445  HP00 486709.51 577447.83
1   91010445  HP02 476777.92 564737.43
2   91010445  HP03 474047.65 572710.40
3   91010445  HP04 473730.49 571679.57
4   91010445  HP05 473308.41 574013.12
5   910104

         tid model   dev_rmse   val_rmse
0   91016149  HP00 1068620.64 1244055.72
1   91016149  HP02 1043151.02 1192354.57
2   91016149  HP03 1042643.46 1189776.35
3   91016149  HP04 1036008.37 1178943.26
4   91016149  HP05 1035766.18 1179804.82
5   91016149  HP06 1032692.97 1177257.60
6   91016149  HP20  825603.26 1016899.60
7   91016149  HP22  787315.53  944417.04
8   91016149  HP23  786939.53  945384.53
9   91016149  HP24  777152.82  929019.65
10  91016149  HP25  776862.14  925885.11
11  91016149  HP26  777127.15  925509.67
12  91016149  HP30  799768.05  993598.67
13  91016149  HP32  761232.98  921083.28
14  91016149  HP33  760002.69  921341.71
15  91016149  HP34  749599.91  901414.97
16  91016149  HP35  748775.26  898061.03
17  91016149  HP36  748626.51  899825.73
0:39:26.445786
         tid model  dev_rmse  val_rmse
0   91016154  HP00 259870.93 270056.09
1   91016154  HP02 256178.12 262072.98
2   91016154  HP03 255700.97 263102.64
3   91016154  HP04 252491.24 256318.85
4   9101615

...

      tid model  dev_rmse  val_rmse
0   BR316  HP00 481716.80 590627.59
1   BR316  HP02 458832.58 588904.58
2   BR316  HP03 458310.84 584488.06
3   BR316  HP04 453080.10 604110.03
4   BR316  HP05 452836.38 609091.30
5   BR316  HP06 452211.51 601223.41
6   BR316  HP20 417037.45 537881.23
7   BR316  HP22 389261.72 555128.06
8   BR316  HP23 388816.27 548614.75
9   BR316  HP24 383354.55 565169.99
10  BR316  HP25 383075.90 569672.82
11  BR316  HP26 380381.05 585696.06
12  BR316  HP30 411219.39 525031.71
13  BR316  HP32 382293.36 546700.12
14  BR316  HP33 381875.72 547037.66
15  BR316  HP34 375887.30 559930.22
16  BR316  HP35 376035.04 562381.34
17  BR316  HP36 374028.88 564967.59
6:09:11.648782
      tid model   dev_rmse   val_rmse
0   BR318  HP00 1083738.44 1130213.36
1   BR318  HP02 1026560.21 1046931.19
2   BR318  HP03 1024306.59 1051257.98
3   BR318  HP04 1010269.09 1032561.43
4   BR318  HP05 1008803.12 1035857.01
5   BR318  HP06 1007501.86 1036159.47
6   BR318  HP20  961686.54  985096.

      tid model   dev_rmse   val_rmse
0   BR339  HP00 1228049.45 1216348.38
1   BR339  HP02 1206995.62 1213653.75
2   BR339  HP03 1206527.53 1210873.02
3   BR339  HP04 1206294.71 1209456.11
4   BR339  HP05 1206261.76 1213322.13
5   BR339  HP06 1203932.09 1208289.60
6   BR339  HP20  776151.93  711989.48
7   BR339  HP22  738010.37  694919.83
8   BR339  HP23  738286.04  694515.21
9   BR339  HP24  738108.89  696587.92
10  BR339  HP25  736914.46  702077.38
11  BR339  HP26  735184.58  704805.97
12  BR339  HP30  719353.42  658191.11
13  BR339  HP32  678433.68  642534.79
14  BR339  HP33  678762.02  642196.35
15  BR339  HP34  678061.55  645002.98
16  BR339  HP35  677643.53  648095.64
17  BR339  HP36  674379.80  655286.21
6:19:50.072647
      tid model  dev_rmse  val_rmse
0   BR340  HP00 571209.45 529898.86
1   BR340  HP02 558695.69 515848.92
2   BR340  HP03 557728.72 516774.41
3   BR340  HP04 554597.16 520615.84
4   BR340  HP05 554369.92 519980.25
5   BR340  HP06 552210.14 515656.51
6   BR340  

      tid model   dev_rmse   val_rmse
0   BR376  HP00 1091853.32 1115200.11
1   BR376  HP02 1060939.67 1099092.39
2   BR376  HP03 1061869.54 1097762.19
3   BR376  HP04 1053609.11 1107035.56
4   BR376  HP05 1053697.90 1107377.25
5   BR376  HP06 1047814.69 1116890.99
6   BR376  HP20  829454.04  916483.89
7   BR376  HP22  781332.38  888181.56
8   BR376  HP23  781950.63  883830.28
9   BR376  HP24  771194.76  886624.70
10  BR376  HP25  770535.35  886670.08
11  BR376  HP26  768801.57  889343.43
12  BR376  HP30  798557.12  870361.91
13  BR376  HP32  747726.68  842961.54
14  BR376  HP33  745787.14  846106.09
15  BR376  HP34  736373.49  843193.58
16  BR376  HP35  734457.97  851669.03
17  BR376  HP36  734356.62  848768.80
6:30:29.709199
      tid model  dev_rmse  val_rmse
0   BR377  HP00 717051.66 726024.80
1   BR377  HP02 704508.02 714724.96
2   BR377  HP03 703877.08 714139.29
3   BR377  HP04 700711.03 717923.03
4   BR377  HP05 699513.88 719679.04
5   BR377  HP06 697383.72 715359.55
6   BR377  

      tid model  dev_rmse  val_rmse
0   BR397  HP00 619291.12 616616.80
1   BR397  HP02 576357.73 639157.68
2   BR397  HP03 570821.26 642054.14
3   BR397  HP04 564293.17 664290.81
4   BR397  HP05 564257.97 669714.65
5   BR397  HP06 562746.75 669026.28
6   BR397  HP20 589465.71 620759.74
7   BR397  HP22 544545.17 653644.54
8   BR397  HP23 539149.65 659700.33
9   BR397  HP24 534331.34 675955.01
10  BR397  HP25 533016.18 687813.28
11  BR397  HP26 530209.23 702085.34
12  BR397  HP30 588332.15 611906.37
13  BR397  HP32 544588.75 646952.92
14  BR397  HP33 537907.14 658390.98
15  BR397  HP34 532377.02 671305.25
16  BR397  HP35 531330.49 684178.55
17  BR397  HP36 528914.80 689490.97
6:41:00.348310
      tid model  dev_rmse  val_rmse
0   BR398  HP00 289912.29 245044.94
1   BR398  HP02 288776.28 245845.89
2   BR398  HP03 288584.60 245059.93
3   BR398  HP04 288281.06 244844.93
4   BR398  HP05 287929.62 244776.59
5   BR398  HP06 287301.90 245164.81
6   BR398  HP20 226320.56 186632.34
7   BR398  HP

KeyboardInterrupt: 

### Single ATM

In [122]:
atm = '91018579'
series = data_train[data_train.tid == atm][['ds','y']].copy()
holidays=HP
i = 0
weekly = scenario.week[i]
monthly = scenario.mon[i]
start=time()
for i in range(scenario.shape[0]):
    d,v= model_prophet(series,holidays,dev_dates, val_dates,
                   weekly=scenario.week[i], monthly=scenario.mon[i])
    print(f'model{scenario.model[i]}:   dev_rmse:{d}    val_rmse:{v}')
print(timedelta(seconds=time()-start))

modelHP00:   dev_rmse:65023.85402626115    val_rmse:62544.60042016784
modelHP02:   dev_rmse:63519.96255130541    val_rmse:61775.855839728785
modelHP03:   dev_rmse:63469.96962782284    val_rmse:62798.06201271611
modelHP04:   dev_rmse:63271.488231253286    val_rmse:63614.74579578053
modelHP05:   dev_rmse:63240.63190895163    val_rmse:64257.580307362885
modelHP06:   dev_rmse:63184.05695892238    val_rmse:63429.765924557905
modelHP20:   dev_rmse:63100.475328195476    val_rmse:61074.91488540868
modelHP22:   dev_rmse:61645.45256718407    val_rmse:61011.9905664003
modelHP23:   dev_rmse:61405.855997176885    val_rmse:61638.440153795025
modelHP24:   dev_rmse:61279.40396157716    val_rmse:61777.14909438259
modelHP25:   dev_rmse:61211.20109247716    val_rmse:61775.80955157328
modelHP26:   dev_rmse:61216.833055880124    val_rmse:62214.33484511408
modelHP30:   dev_rmse:62787.464668768545    val_rmse:60705.16712954916
modelHP32:   dev_rmse:61227.80467776674    val_rmse:60821.17506901428
modelHP33:  

In [128]:
atm = 'BR324'
series = data_train[data_train.tid == atm][['ds','y']].copy()
holidays=HP
i = 0
weekly = scenario.week[i]
monthly = scenario.mon[i]
start=time()
for i in range(scenario.shape[0]):
    d,v= model_prophet(series,holidays,dev_dates, val_dates,
                   weekly=scenario.week[i], monthly=scenario.mon[i])
    print(f'model{scenario.model[i]}:   dev_rmse:{d}    val_rmse:{v}')
print(timedelta(seconds=time()-start))

modelHP00:   dev_rmse:178832.99991624343    val_rmse:171881.06577271942
modelHP02:   dev_rmse:175829.77257917632    val_rmse:174719.90116092126
modelHP03:   dev_rmse:175619.5685729972    val_rmse:174620.90607456202
modelHP04:   dev_rmse:175203.3701339818    val_rmse:176277.52874055458
modelHP05:   dev_rmse:175010.644588449    val_rmse:175856.3479843491
modelHP06:   dev_rmse:174692.32772506922    val_rmse:174708.68273922114
modelHP20:   dev_rmse:158305.00424032193    val_rmse:141450.50859721578
modelHP22:   dev_rmse:154844.93918730557    val_rmse:145437.85635162695
modelHP23:   dev_rmse:154531.41672478893    val_rmse:145881.3186816657
modelHP24:   dev_rmse:153911.03429120645    val_rmse:147349.50757336727
modelHP25:   dev_rmse:153813.9555346343    val_rmse:147311.03854240186
modelHP26:   dev_rmse:153312.7765273328    val_rmse:148650.08948617897
modelHP30:   dev_rmse:152596.3253216912    val_rmse:136541.66657663262
modelHP32:   dev_rmse:148939.56016728864    val_rmse:140677.67069861485
m

### Parallel Scenario

In [137]:
atm = 'BR324'
series = data_train[data_train.tid == atm][['ds','y']].copy()
holidays=HP

dbweek = db.from_sequence([scenario.week[i] for i in range(scenario.shape[0])])
dbmon = db.from_sequence([scenario.mon[i] for i in range(scenario.shape[0])])

In [144]:
dbmaster = db.map(train,series,holidays,dev_dates, val_dates,weekly=dbweek,monthly=dbmon)

In [146]:
start=time()
model = dbmaster.compute()
print(timedelta(seconds=time()-start))

0:00:54.890126


### Parallel ATM - FOREVER

In [161]:
start=time()
model = par_scen(series,atm)
print(timedelta(seconds=time()-start))

0:02:54.932021


In [167]:
atms

array(['91010033', '91010053', '91010068', '91010075', '91010090',
       '91010101', '91010125', '91010126', '91010127', '91010133',
       '91010135', '91010145', '91010151', '91010152', '91010155',
       '91010156', '91010157', '91010158', '91010162', '91010211',
       '91010264', '91010265', '91010301', '91010320', '91010330',
       '91010358', '91010363', '91010366', '91010371', '91010376',
       '91010383', '91010391', '91010392', '91010443', '91010445',
       '91010465', '91010476', '91010491', '91016061', '91016062',
       '91016063', '91016065', '91016131', '91016147', '91016149',
       '91016154', '91016162', '91016229', '91016245', '91016253',
       '91016261', '91016265', '91016266', '91016267', '91016268',
       '91016269', '91016270', '91016272', '91016274', '91016283',
       '91016300', '91016311', '91016323', '91016325', '91016326',
       '91016369', '91016370', '91016408', '91016409', '91016488',
       '91016491', '91016617', '91016620', '91016621', '910167

In [169]:
start = time()
dbseries = db.from_sequence([data_train[data_train.tid == atm][['ds','y']].copy() for atm in atms[:24]])
dbtid= db.from_sequence(atms[:24])
dbmaster = db.map(par_scen, dbseries, dbtid)
model = dbmaster.compute()
print(timedelta(seconds=time()-start))

KeyboardInterrupt: 

In [165]:
a

,model,dev_rmse,val_rmse
0,HP00,178833.00,171881.07
1,HP02,175829.77,174719.90
2,HP03,175619.57,174620.91
3,HP04,175203.37,176277.53
4,HP05,175010.64,175856.35
5,HP06,174692.33,174708.68
6,HP20,158305.00,141450.51
7,HP22,154844.94,145437.86
8,HP23,154531.42,145881.32
9,HP24,153911.03,147349.51


### Run all ATMs

In [23]:
start = time()
for atm in atms[:1]:
    atm = '91018579'
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


91018579
model: HP00  rmse: 61915.82973079349
model: HP02  rmse: 60596.55302156461
model: HP03  rmse: 60942.31008111984
model: HP04  rmse: 60780.479617553865
model: HP05  rmse: 61141.4744513326
model: HP06  rmse: 61426.89199678496
model: HP20  rmse: 59450.88486359512
model: HP22  rmse: 58219.351276451525
model: HP23  rmse: 58668.86791255217
model: HP24  rmse: 58444.84319118447
model: HP25  rmse: 58830.94363042807
model: HP26  rmse: 59004.657420200754
model: HP30  rmse: 59528.12037790119
model: HP32  rmse: 58293.4474216982
model: HP33  rmse: 58764.68115463953
model: HP34  rmse: 58500.84862656673
model: HP35  rmse: 58828.97509437736
model: HP36  rmse: 59024.94662242007
0:12:39.253158


In [4]:
start = time()
for atm in atms[20:30]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


BR159
model: HP00  rmse: 1155173.253135945
model: HP02  rmse: 1120355.5418058191
model: HP03  rmse: 1125361.065040681
model: HP04  rmse: 1159344.130432704
model: HP05  rmse: 1164343.8398152094
model: HP06  rmse: 1165177.5100449608
model: HP20  rmse: 1094027.9425948183
model: HP22  rmse: 1064601.829465189
model: HP23  rmse: 1067564.9314907875
model: HP24  rmse: 1079476.1504817512
model: HP25  rmse: 1078831.5590779318
model: HP26  rmse: 1081412.5655150344
model: HP30  rmse: 1098771.478691766
model: HP32  rmse: 1069923.5305825353
model: HP33  rmse: 1072523.2844048368
model: HP34  rmse: 1085273.2950016237
model: HP35  rmse: 1084628.3026237064
model: HP36  rmse: 1088420.4528774878
0:03:32.286452

BR319
model: HP00  rmse: 1178666.9228666222
model: HP02  rmse: 1073886.49392609
model: HP03  rmse: 1077375.1943505364
model: HP04  rmse: 1102522.970296644
model: HP05  rmse: 1105030.7674793531
model: HP06  rmse: 1096074.2660998602
model: HP20  rmse: 1143276.7083193318
model: HP22  rmse: 1030209.02

In [4]:
start = time()
for atm in atms[30:40]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


BR077
model: HP00  rmse: 1050102.5281321101
model: HP02  rmse: 953378.7155784646
model: HP03  rmse: 971123.752447076
model: HP04  rmse: 1004090.0107124847
model: HP05  rmse: 1005911.633448739
model: HP06  rmse: 995331.0861118428
model: HP20  rmse: 978561.8341418764
model: HP22  rmse: 880350.430181264
model: HP23  rmse: 894337.578428938
model: HP24  rmse: 908283.9332328822
model: HP25  rmse: 907920.8122008827
model: HP26  rmse: 900012.7126497334
model: HP30  rmse: 988899.9249423798
model: HP32  rmse: 892788.9705481051
model: HP33  rmse: 906520.9092711672
model: HP34  rmse: 922876.9212971518
model: HP35  rmse: 921014.8751576503
model: HP36  rmse: 913109.3674552651
0:03:32.649983

BR835
model: HP00  rmse: 927632.2581710827
model: HP02  rmse: 853625.4551292931
model: HP03  rmse: 860945.6253207272
model: HP04  rmse: 874496.212328124
model: HP05  rmse: 872941.7600797793
model: HP06  rmse: 870394.8125982742
model: HP20  rmse: 835444.8181282433
model: HP22  rmse: 746883.0879567002
model: HP23

In [4]:
start = time()
for atm in atms[40:50]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


91017028
model: HP00  rmse: 757057.9122961547
model: HP02  rmse: 697077.0021912791
model: HP03  rmse: 710529.8083746545
model: HP04  rmse: 746323.4686085087
model: HP05  rmse: 741057.5575721544
model: HP06  rmse: 739503.0893102445
model: HP20  rmse: 754479.2763743547
model: HP22  rmse: 694290.7469303403
model: HP23  rmse: 703308.160207017
model: HP24  rmse: 720554.4392160783
model: HP25  rmse: 718263.768393855
model: HP26  rmse: 717201.7424832897
model: HP30  rmse: 764044.2616498141
model: HP32  rmse: 702650.7642725061
model: HP33  rmse: 712947.7731347186
model: HP34  rmse: 729401.7595780917
model: HP35  rmse: 728551.3292833111
model: HP36  rmse: 726033.0646853267
0:03:13.180585

BR894
model: HP00  rmse: 1244200.757850247
model: HP02  rmse: 1169866.6655965836
model: HP03  rmse: 1181059.23375449
model: HP04  rmse: 1149203.9557726216
model: HP05  rmse: 1146412.8879650466
model: HP06  rmse: 1126151.3879332687
model: HP20  rmse: 1099353.978422826
model: HP22  rmse: 1010474.641072418
model

In [4]:
start = time()
for atm in atms[60:70]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


BR419
model: HP00  rmse: 946311.6433542137
model: HP02  rmse: 884703.6029449885
model: HP03  rmse: 893025.8000503778
model: HP04  rmse: 910416.6781912396
model: HP05  rmse: 912488.3692298796
model: HP06  rmse: 900095.6508136083
model: HP20  rmse: 857978.5764932213
model: HP22  rmse: 780018.2167262172
model: HP23  rmse: 787580.6968257503
model: HP24  rmse: 798261.0628128091
model: HP25  rmse: 799586.3533694398
model: HP26  rmse: 785705.3697801295
model: HP30  rmse: 876870.5224300503
model: HP32  rmse: 805096.7313510346
model: HP33  rmse: 811031.3831217161
model: HP34  rmse: 822518.6745192183
model: HP35  rmse: 824475.2618507051
model: HP36  rmse: 811021.0319961205
0:03:31.481356

BR339
model: HP00  rmse: 809705.3214007864
model: HP02  rmse: 800283.4418192079
model: HP03  rmse: 797190.8973250648
model: HP04  rmse: 793228.276867304
model: HP05  rmse: 798300.5243679419
model: HP06  rmse: 799291.0211471758
model: HP20  rmse: 545364.08288916
model: HP22  rmse: 502872.63287707325
model: HP23

In [4]:
start = time()
for atm in atms[90:100]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


BR144
model: HP00  rmse: 894747.8771140062
model: HP02  rmse: 873891.0710401721
model: HP03  rmse: 875420.0859651834
model: HP04  rmse: 872891.4672049856
model: HP05  rmse: 872441.0220006322
model: HP06  rmse: 874740.3844190801
model: HP20  rmse: 659877.1756727167
model: HP22  rmse: 608568.7216837787
model: HP23  rmse: 614554.365810954
model: HP24  rmse: 612839.6158529642
model: HP25  rmse: 612422.9882031421
model: HP26  rmse: 603811.1676903063
model: HP30  rmse: 664081.5921135889
model: HP32  rmse: 615126.2026133857
model: HP33  rmse: 617155.4436441633
model: HP34  rmse: 616294.4911453571
model: HP35  rmse: 617148.3417663367
model: HP36  rmse: 608135.3407511374
0:03:33.734984

BR877
model: HP00  rmse: 308335.54691026936
model: HP02  rmse: 311071.59462601104
model: HP03  rmse: 311246.9442572497
model: HP04  rmse: 336269.3071352798
model: HP05  rmse: 339066.95226314093
model: HP06  rmse: 338691.11342431454
model: HP20  rmse: 257522.41261581602
model: HP22  rmse: 260801.14491954786
mode

In [4]:
start = time()
for atm in atms[110:120]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


BR411
model: HP00  rmse: 895879.0708075454
model: HP02  rmse: 865623.2885220741
model: HP03  rmse: 864916.939575608
model: HP04  rmse: 882432.4953492306
model: HP05  rmse: 885625.1214815538
model: HP06  rmse: 877439.6523567934
model: HP20  rmse: 883870.7706655336
model: HP22  rmse: 850210.2103181774
model: HP23  rmse: 848943.5941832407
model: HP24  rmse: 853274.0214443033
model: HP25  rmse: 852957.197022336
model: HP26  rmse: 845483.2052905256
model: HP30  rmse: 886736.6216158308
model: HP32  rmse: 853320.6071208676
model: HP33  rmse: 852618.8077295825
model: HP34  rmse: 857465.2214495114
model: HP35  rmse: 857089.5833002093
model: HP36  rmse: 849261.3622992026
0:03:25.008962

BR309
model: HP00  rmse: 682537.1078597653
model: HP02  rmse: 633989.3466544608
model: HP03  rmse: 629844.3671072518
model: HP04  rmse: 623718.6987451222
model: HP05  rmse: 623000.8125823343
model: HP06  rmse: 623696.1255322181
model: HP20  rmse: 584321.463359137
model: HP22  rmse: 513014.92283466726
model: HP23

In [4]:
start = time()
for atm in atms[120:140]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


BR331
model: HP00  rmse: 660142.1098372164
model: HP02  rmse: 638776.0454303047
model: HP03  rmse: 649553.0714734249
model: HP04  rmse: 652503.5544498534
model: HP05  rmse: 654206.6064561353
model: HP06  rmse: 656644.1127473613
model: HP20  rmse: 522744.6149588165
model: HP22  rmse: 487349.268269001
model: HP23  rmse: 501140.80315823236
model: HP24  rmse: 505267.9388569938
model: HP25  rmse: 508504.24648674735
model: HP26  rmse: 506947.6232518304
model: HP30  rmse: 530804.5339382191
model: HP32  rmse: 499028.6976954314
model: HP33  rmse: 510338.0465480242
model: HP34  rmse: 515431.81282233197
model: HP35  rmse: 518128.7674463019
model: HP36  rmse: 517662.75366914214
0:07:07.121030

BR351
model: HP00  rmse: 918104.2406651267
model: HP02  rmse: 848257.36784612
model: HP03  rmse: 853961.7545789124
model: HP04  rmse: 862711.9451782026
model: HP05  rmse: 860005.584301784
model: HP06  rmse: 849721.8644277973
model: HP20  rmse: 907162.0032501763
model: HP22  rmse: 834840.9139131128
model: HP


BR335
model: HP00  rmse: 688728.9178086993
model: HP02  rmse: 657992.8179385045
model: HP03  rmse: 668996.5251800023
model: HP04  rmse: 668747.9775723319
model: HP05  rmse: 669141.714298085
model: HP06  rmse: 643192.3372421752
model: HP20  rmse: 630707.7100823835
model: HP22  rmse: 591381.3755619734
model: HP23  rmse: 603166.433795702
model: HP24  rmse: 601051.653958344
model: HP25  rmse: 603605.4290247845
model: HP26  rmse: 573314.3949717621
model: HP30  rmse: 654240.7073666237
model: HP32  rmse: 616485.1994502104
model: HP33  rmse: 627987.6457626572
model: HP34  rmse: 626942.1647919203
model: HP35  rmse: 629389.7517898185
model: HP36  rmse: 599713.3298672885
1:24:04.486635

91016267
model: HP00  rmse: 1041458.4707180908
model: HP02  rmse: 988469.734480509
model: HP03  rmse: 993426.1039193413
model: HP04  rmse: 992579.5908905643
model: HP05  rmse: 984757.4883342489
model: HP06  rmse: 970101.5434623219
model: HP20  rmse: 960909.3444794044
model: HP22  rmse: 889554.545870551
model: HP2

In [4]:
start = time()
for atm in atms[160:180]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


91018361
model: HP00  rmse: 334486.7262201482
model: HP02  rmse: 328262.5983603709
model: HP03  rmse: 334022.8025601575
model: HP04  rmse: 332920.3389655835
model: HP05  rmse: 334718.9926278313
model: HP06  rmse: 331087.74024468276
model: HP20  rmse: 347155.16806253296
model: HP22  rmse: 340372.7772387986
model: HP23  rmse: 344995.3492128433
model: HP24  rmse: 338508.2255256705
model: HP25  rmse: 342316.1955868862
model: HP26  rmse: 339543.54159288167
model: HP30  rmse: 353730.0272094276
model: HP32  rmse: 348223.7320619733
model: HP33  rmse: 352356.8767683944
model: HP34  rmse: 346085.66149718745
model: HP35  rmse: 349400.60891181557
model: HP36  rmse: 346450.80800048384
0:03:32.398691

BR494
model: HP00  rmse: 261957.40162993688
model: HP02  rmse: 255604.9289222592
model: HP03  rmse: 258369.93736227727
model: HP04  rmse: 257419.54121714857
model: HP05  rmse: 259861.26835872608
model: HP06  rmse: 253938.73443739992
model: HP20  rmse: 278807.4526749989
model: HP22  rmse: 275752.325835


91018573
model: HP00  rmse: 882265.1365595218
model: HP02  rmse: 870498.557166875
model: HP03  rmse: 867417.8021761485
model: HP04  rmse: 879528.3416797206
model: HP05  rmse: 888822.912571964
model: HP06  rmse: 889649.5382817734
model: HP20  rmse: 689271.6953406205
model: HP22  rmse: 684590.7604181884
model: HP23  rmse: 680351.6852856778
model: HP24  rmse: 683457.5497505328
model: HP25  rmse: 687027.757844379
model: HP26  rmse: 688522.9164951212
model: HP30  rmse: 686327.1615788707
model: HP32  rmse: 681221.0287316445
model: HP33  rmse: 676728.6261345025
model: HP34  rmse: 681353.4700810192
model: HP35  rmse: 684843.9698177666
model: HP36  rmse: 686141.3807896686
0:41:05.401820

91017893
model: HP00  rmse: 610639.8096205278
model: HP02  rmse: 619909.5418836494
model: HP03  rmse: 618521.5410366936
model: HP04  rmse: 658981.2458009942
model: HP05  rmse: 651050.7938094863
model: HP06  rmse: 653750.1021351796
model: HP20  rmse: 551618.0596147997
model: HP22  rmse: 552672.6243051045
model:

In [4]:
start = time()
for atm in atms[280:300]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


91018467
model: HP00  rmse: 603705.3064811128
model: HP02  rmse: 599972.9236222917
model: HP03  rmse: 600856.470423687
model: HP04  rmse: 611193.4312268585
model: HP05  rmse: 598382.392216883
model: HP06  rmse: 594641.0392061134
model: HP20  rmse: 461352.22836486297
model: HP22  rmse: 456635.95940699894
model: HP23  rmse: 457648.7037176608
model: HP24  rmse: 461791.08017030085
model: HP25  rmse: 453794.4853218726
model: HP26  rmse: 453150.6463014369
model: HP30  rmse: 446297.4425006819
model: HP32  rmse: 441467.4879523845
model: HP33  rmse: 440958.90419481986
model: HP34  rmse: 445698.3588868013
model: HP35  rmse: 439092.55543807975
model: HP36  rmse: 434049.42190936947
0:03:34.581727

BR298
model: HP00  rmse: 233746.4449682002
model: HP02  rmse: 217555.9695029881
model: HP03  rmse: 219338.7921907657
model: HP04  rmse: 224175.73029411738
model: HP05  rmse: 227307.91618013862
model: HP06  rmse: 227780.0457963592
model: HP20  rmse: 209342.55736108177
model: HP22  rmse: 187341.334814989



BR466
model: HP00  rmse: 621169.306322008
model: HP02  rmse: 562056.8298410428
model: HP03  rmse: 571357.5520599613
model: HP04  rmse: 574329.3388052235
model: HP05  rmse: 577384.7684723099
model: HP06  rmse: 569189.0772420674
model: HP20  rmse: 614728.4795712784
model: HP22  rmse: 546285.1673300818
model: HP23  rmse: 556067.3511267018
model: HP24  rmse: 556748.3116767908
model: HP25  rmse: 559775.2074288514
model: HP26  rmse: 550366.0396059679
model: HP30  rmse: 627012.7475078077
model: HP32  rmse: 561233.7592112066
model: HP33  rmse: 569866.2926637599
model: HP34  rmse: 571083.8928921326
model: HP35  rmse: 575707.9253368516
model: HP36  rmse: 564866.2704100456
0:41:02.223140

91017006
model: HP00  rmse: 445290.4219331519
model: HP02  rmse: 424119.40966097283
model: HP03  rmse: 421193.60986702464
model: HP04  rmse: 434266.35102208244
model: HP05  rmse: 433557.60352234857
model: HP06  rmse: 432105.5666789631
model: HP20  rmse: 459053.5072018398
model: HP22  rmse: 444002.2324488406
mod

In [4]:
start = time()
for atm in atms[400:420]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


91017977
model: HP00  rmse: 488278.72344501846
model: HP02  rmse: 461222.2067153795
model: HP03  rmse: 453571.6280442737
model: HP04  rmse: 475675.0755915269
model: HP05  rmse: 477132.8135031456
model: HP06  rmse: 479878.58963609627
model: HP20  rmse: 445643.02734725585
model: HP22  rmse: 416980.64055945224
model: HP23  rmse: 409117.0619810765
model: HP24  rmse: 407204.1530695865
model: HP25  rmse: 405868.9433708031
model: HP26  rmse: 412139.471138042
model: HP30  rmse: 440396.04431129195
model: HP32  rmse: 409693.0447846547
model: HP33  rmse: 402155.29907993536
model: HP34  rmse: 400716.2138233157
model: HP35  rmse: 399881.77914701414
model: HP36  rmse: 404622.9480507771
0:03:10.671696

91019866
model: HP00  rmse: 406263.15125689545
model: HP02  rmse: 370121.5574445375
model: HP03  rmse: 375686.28681775794
model: HP04  rmse: 382924.2139075395
model: HP05  rmse: 380970.22548331413
model: HP06  rmse: 375760.1851644268
model: HP20  rmse: 383304.8043750059
model: HP22  rmse: 343750.72822


91010127
model: HP00  rmse: 316153.01587490115
model: HP02  rmse: 327001.5839222992
model: HP03  rmse: 329174.3290906828
model: HP04  rmse: 324521.44350958616
model: HP05  rmse: 325654.30667263176
model: HP06  rmse: 321987.2523400761
model: HP20  rmse: 343002.63574869296
model: HP22  rmse: 348463.85623108514
model: HP23  rmse: 349920.2897092876
model: HP24  rmse: 330073.9392725642
model: HP25  rmse: 330643.41108237166
model: HP26  rmse: 326580.1713117501
model: HP30  rmse: 352440.1632040019
model: HP32  rmse: 356421.80700848985
model: HP33  rmse: 357953.7984600183
model: HP34  rmse: 338999.07157447556
model: HP35  rmse: 339333.42874020257
model: HP36  rmse: 335268.41734021634
0:41:31.884903

BR467
model: HP00  rmse: 361892.9044813759
model: HP02  rmse: 333058.51153000287
model: HP03  rmse: 338818.86260660336
model: HP04  rmse: 342006.6846128032
model: HP05  rmse: 342010.27900985087
model: HP06  rmse: 341596.25579580845
model: HP20  rmse: 330890.1836618349
model: HP22  rmse: 293511.001

In [4]:
start = time()
for atm in atms[460:480]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


BR479
model: HP00  rmse: 210822.0566285378
model: HP02  rmse: 204690.46297740703
model: HP03  rmse: 203959.38217981442
model: HP04  rmse: 206999.8541754332
model: HP05  rmse: 206068.33166955775
model: HP06  rmse: 205239.17438948754
model: HP20  rmse: 189174.5597508011
model: HP22  rmse: 182634.66108449508
model: HP23  rmse: 182033.90335933786
model: HP24  rmse: 183962.5244816503
model: HP25  rmse: 182510.51580356754
model: HP26  rmse: 181715.4074502325
model: HP30  rmse: 192522.10141918252
model: HP32  rmse: 187278.45276103518
model: HP33  rmse: 187080.4733894971
model: HP34  rmse: 189031.41656349416
model: HP35  rmse: 188147.80191905922
model: HP36  rmse: 186657.13470492867
0:03:24.317521

91016323
model: HP00  rmse: 300542.3467822953
model: HP02  rmse: 300685.19233835494
model: HP03  rmse: 299911.9974342105
model: HP04  rmse: 304700.8259766124
model: HP05  rmse: 301647.26215955656
model: HP06  rmse: 298426.2178029118
model: HP20  rmse: 290127.01522166986
model: HP22  rmse: 284275.87


91010126
model: HP00  rmse: 299206.0267419816
model: HP02  rmse: 267979.38267057855
model: HP03  rmse: 272484.6728199356
model: HP04  rmse: 252653.98677908842
model: HP05  rmse: 252856.75886849532
model: HP06  rmse: 253971.83015596567
model: HP20  rmse: 289383.5464963974
model: HP22  rmse: 253570.27828427352
model: HP23  rmse: 257984.81490039665
model: HP24  rmse: 243460.2013495007
model: HP25  rmse: 242078.69746314076
model: HP26  rmse: 240517.41135635762
model: HP30  rmse: 290837.4267581988
model: HP32  rmse: 255179.75264544887
model: HP33  rmse: 259326.98934275535
model: HP34  rmse: 244984.1714223479
model: HP35  rmse: 243925.01944011476
model: HP36  rmse: 241593.8083514226
0:41:38.756214

BR831
model: HP00  rmse: 418473.4631663685
model: HP02  rmse: 415642.7584871169
model: HP03  rmse: 412198.9373428716
model: HP04  rmse: 416316.0551004829
model: HP05  rmse: 415614.1053498975
model: HP06  rmse: 420003.89276560355
model: HP20  rmse: 393473.5352459628
model: HP22  rmse: 389430.14570

In [4]:
start = time()
for atm in atm2[80:100]:
    series = data_train[data_train.tid == atm][['ds','y']].copy()
    model_ = edd.parallel_model_prophet_oneholi(
                    series,scenario,HP,dev_length, period, horizon)
    with open(out + '_'+atm+'.pkl', 'wb') as f: pickle.dump(model_, f)
        
    print("\n{}".format(atm))
    for i in range(scenario.shape[0]):
        print("model: {}  rmse: {}".format(scenario.model[i], model_[i][2].rmse.mean()))

    print(timedelta(seconds=time()-start))


91018923
model: HP00  rmse: 464513.7247391073
model: HP02  rmse: 452964.6733768204
model: HP03  rmse: 451979.1043373924
model: HP04  rmse: 467268.3506591315
model: HP05  rmse: 463635.1409890498
model: HP06  rmse: 460891.1932333556
model: HP20  rmse: 411846.4032828996
model: HP22  rmse: 391554.54688469885
model: HP23  rmse: 391854.5900970426
model: HP24  rmse: 404557.81522086234
model: HP25  rmse: 402206.33498805005
model: HP26  rmse: 396545.3538365806
model: HP30  rmse: 407584.0884847149
model: HP32  rmse: 387312.0431569047
model: HP33  rmse: 386867.88591948326
model: HP34  rmse: 400583.38551958144
model: HP35  rmse: 397782.3025333838
model: HP36  rmse: 392547.43025812006
0:03:20.760805

91016530
model: HP00  rmse: 430788.7202334982
model: HP02  rmse: 427170.16698159976
model: HP03  rmse: 428221.6897000399
model: HP04  rmse: 434676.5138879862
model: HP05  rmse: 433600.512631246
model: HP06  rmse: 431902.204158103
model: HP20  rmse: 397053.12855335604
model: HP22  rmse: 394401.24477842


91018912
model: HP00  rmse: 329454.8661654238
model: HP02  rmse: 331493.2437772691
model: HP03  rmse: 340304.5467114559
model: HP04  rmse: 338922.4475271053
model: HP05  rmse: 339839.82375351863
model: HP06  rmse: 339654.5394054655
model: HP20  rmse: 258465.28321263447
model: HP22  rmse: 257173.95768076528
model: HP23  rmse: 269793.1166494466
model: HP24  rmse: 271554.7708756343
model: HP25  rmse: 273030.4224095244
model: HP26  rmse: 271914.6699024655
model: HP30  rmse: 266019.7483416171
model: HP32  rmse: 264998.4028217481
model: HP33  rmse: 276530.0887432511
model: HP34  rmse: 278830.4177343427
model: HP35  rmse: 279628.1114970144
model: HP36  rmse: 278689.6112482637
0:41:25.188939

91016392
model: HP00  rmse: 692316.6518137254
model: HP02  rmse: 662712.8284981064
model: HP03  rmse: 655886.0498800094
model: HP04  rmse: 652619.330992106
model: HP05  rmse: 658125.6964022043
model: HP06  rmse: 646696.571752193
model: HP20  rmse: 689944.8876860605
model: HP22  rmse: 655242.8610142899
mo